In [78]:
from pykrx import stock
import pandas as pd
import mplfinance as mpf
import numpy as np
import plotly.graph_objects as go
import plotly.subplots as ms
import plotly.express as px

In [79]:
# pip install pykrx
# pip install mplfinance

In [80]:
# 전체 종목코드와 종목명 가져오기
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

In [81]:
name_list = list(stock_list['종목명'])

dff = pd.DataFrame()
stock_from = "20190104" # 2249~2301
stock_to = "20191206"

for name in name_list:
    stock_name = name
    ticker = stock_list.loc[stock_list['종목명']== stock_name, '종목코드']
    df = stock.get_market_ohlcv_by_date(fromdate=stock_from, todate=stock_to, ticker=ticker)
# 칼럼명을 영문명으로 변경
    df['name'] = name
    df["종가"]=df["종가"].apply(pd.to_numeric,errors="coerce")
    dff = pd.concat([dff,df])
dff = dff.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})

In [82]:
# 특정 날이 존재하는 종목 리스트 뽑기 
import numpy as np

df = dff

dname = df.loc[stock_from]['name']
dname = list(dname.values)
dff = pd.DataFrame()

for name in dname:
    stock_name = name
    ticker = stock_list.loc[stock_list['종목명']== stock_name, '종목코드']
    df = stock.get_market_ohlcv_by_date(fromdate=stock_from, todate=stock_to, ticker=ticker)
    df = df.assign(종목명=stock_name)
# 칼럼명을 영문명으로 변경
    df["종가"]=df["종가"].apply(pd.to_numeric,errors="coerce")
    dff = pd.concat([dff,df])
dff = dff.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})

In [83]:
# 지표 데이터 프레임 만들기
df = dff

df['ma20'] = df['Close'].rolling(window=20).mean() # 20일 이동평균
df['stddev'] = df['Close'].rolling(window=20).std() # 20일 이동표준편차
df['upper'] = df['ma20'] + 2*df['stddev'] # 상단밴드
df['lower'] = df['ma20'] - 2*df['stddev'] # 하단밴드

df['ma12'] = df['Close'].rolling(window=12).mean() # 12일 이동평균
df['ma26'] = df['Close'].rolling(window=26).mean() # 26일 이동평균
df['MACD'] = df['ma12'] - df['ma26']    # MACD
df['MACD_Signal'] = df['MACD'].rolling(window=9).mean() # MACD Signal(MACD 9일 이동평균)
df['MACD_Oscil'] = df['MACD'] - df['MACD_Signal']   #MACD 오실레이터

df['ndays_high'] = df['High'].rolling(window=14, min_periods=1).max()    # 14일 중 최고가
df['ndays_low'] = df['Low'].rolling(window=14, min_periods=1).min()      # 14일 중 최저가
df['fast_k'] = (df['Close'] - df['ndays_low']) / (df['ndays_high'] - df['ndays_low']) * 100  # Fast %K 구하기
df['slow_d'] = df['fast_k'].rolling(window=3).mean()    # Slow %D 구하기

#RSI 구하기
U = np.where(df['Close'].diff(1) > 0, df['Close'].diff(1), 0)
D = np.where(df['Close'].diff(1) < 0, df['Close'].diff(1) *(-1), 0)
AU = pd.DataFrame(U, index=df.index).rolling(window=14).mean()
AD = pd.DataFrame(D, index=df.index).rolling(window=14).mean()
RSI = AU / (AD+AU) *100
df['RSI'] = RSI

# cci 구하기
df['M'] = 0.0
df['m'] = 0.0
df['D'] = 0.0
df['CCI'] = 0.0

df['M'] = (df['High']+df['Low']+df['Close'])/3

df['m'] = df['M'].rolling(window=20).mean()
df['D'] = abs(df['M'] - df['m']).rolling(window=20).mean() # abs
df['CCI'] = (df['M']-df['m'])/(0.015*df['D'])

df

,Open,High,Low,Close,Volume,종목명,ma20,stddev,upper,lower,...,MACD_Oscil,ndays_high,ndays_low,fast_k,slow_d,RSI,M,m,D,CCI
날짜,,,,,,,,,,,,,,,,,,,,,
2019-01-04,2180,2395,2130,2240,180908,3S,NaN,NaN,NaN,NaN,...,NaN,2395.0,2130.0,41.509434,NaN,NaN,2255.000000,NaN,NaN,NaN
2019-01-07,2260,2315,2225,2260,77610,3S,NaN,NaN,NaN,NaN,...,NaN,2395.0,2130.0,49.056604,NaN,NaN,2266.666667,NaN,NaN,NaN
2019-01-08,2230,2265,2200,2250,39095,3S,NaN,NaN,NaN,NaN,...,NaN,2395.0,2130.0,45.283019,45.283019,NaN,2238.333333,NaN,NaN,NaN
2019-01-09,2260,2260,2210,2230,35077,3S,NaN,NaN,NaN,NaN,...,NaN,2395.0,2130.0,37.735849,44.025157,NaN,2233.333333,NaN,NaN,NaN
2019-01-10,2205,2245,2205,2230,20763,3S,NaN,NaN,NaN,NaN,...,NaN,2395.0,2130.0,37.735849,40.251572,NaN,2226.666667,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-02,11946,12397,11846,12197,42673,힘스,12260.75,486.426700,13233.603400,11287.896600,...,235.332621,13599.0,11420.0,35.658559,32.653975,54.084335,12146.666667,12281.683333,419.104167,-21.477026
2019-12-03,12171,12572,11721,12447,51017,힘스,12272.00,488.082392,13248.164784,11295.835216,...,166.462251,13599.0,11570.0,43.223263,34.340445,60.095159,12246.666667,12277.100000,414.291667,-4.897248
2019-12-04,12572,13248,12547,12748,182189,힘스,12309.55,494.656444,13298.862888,11320.237112,...,67.440171,13599.0,11670.0,55.883878,44.921900,55.491463,12847.666667,12317.983333,421.424167,83.792590


In [84]:
# 보조지표 계산에 필요한 NAN값 제거 
df = df.reset_index(drop=False, inplace = False)

a = [0]*38
b = [1]*(len(df[df['종목명'] == dname[0]])-38)
print(b)
c = (a+b)*len(dname)
c = pd.DataFrame(c)

df['bool'] = c
df = df[df['bool'] == 1]

df = df.reset_index(drop=False, inplace = False)
df

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,stddev,...,ndays_high,ndays_low,fast_k,slow_d,RSI,M,m,D,CCI,bool
0,38,2019-03-05,2400,2400,2280,2310,83997,3S,2361.00,88.177453,...,2465.0,2170.0,47.457627,42.187368,34.210526,2330.000000,2366.083333,92.920833,-25.888226,1
1,39,2019-03-06,2335,2350,2305,2315,40146,3S,2352.75,84.081336,...,2400.0,2170.0,63.043478,53.251612,39.130435,2323.333333,2358.750000,90.241667,-26.164312,1
2,40,2019-03-07,2320,2350,2280,2315,83304,3S,2344.00,77.927091,...,2400.0,2170.0,63.043478,57.848195,42.857143,2315.000000,2350.833333,87.962500,-27.158038,1
3,41,2019-03-08,2280,2310,2275,2305,80819,3S,2333.75,67.762647,...,2400.0,2170.0,58.695652,61.594203,44.262295,2296.666667,2340.500000,85.175000,-34.308450,1
4,42,2019-03-11,2280,2340,2265,2340,50035,3S,2325.75,55.423608,...,2400.0,2170.0,73.913043,65.217391,52.307692,2315.000000,2330.833333,81.412500,-12.965522,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426498,511352,2019-12-02,11946,12397,11846,12197,42673,힘스,12260.75,486.426700,...,13599.0,11420.0,35.658559,32.653975,54.084335,12146.666667,12281.683333,419.104167,-21.477026,1
426499,511353,2019-12-03,12171,12572,11721,12447,51017,힘스,12272.00,488.082392,...,13599.0,11570.0,43.223263,34.340445,60.095159,12246.666667,12277.100000,414.291667,-4.897248,1
426500,511354,2019-12-04,12572,13248,12547,12748,182189,힘스,12309.55,494.656444,...,13599.0,11670.0,55.883878,44.921900,55.491463,12847.666667,12317.983333,421.424167,83.792590,1
426501,511355,2019-12-05,12848,12898,12247,12447,66915,힘스,12348.35,471.841441,...,13599.0,11670.0,40.279938,46.462359,45.728888,12530.666667,12356.400000,399.865833,29.054190,1


In [85]:
# 거래량 0인적 있는 데이터 제거 

name_list = list(df[df['Volume'] == 0]['종목명'].unique())

# name_list는 volume = 0 인적 잇는것들 
for n in name_list:
    df = df[df['종목명'] != n ]
    
df = df.reset_index(drop=False, inplace = False)
df

,level_0,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,...,ndays_high,ndays_low,fast_k,slow_d,RSI,M,m,D,CCI,bool
0,0,38,2019-03-05,2400,2400,2280,2310,83997,3S,2361.00,...,2465.0,2170.0,47.457627,42.187368,34.210526,2330.000000,2366.083333,92.920833,-25.888226,1
1,1,39,2019-03-06,2335,2350,2305,2315,40146,3S,2352.75,...,2400.0,2170.0,63.043478,53.251612,39.130435,2323.333333,2358.750000,90.241667,-26.164312,1
2,2,40,2019-03-07,2320,2350,2280,2315,83304,3S,2344.00,...,2400.0,2170.0,63.043478,57.848195,42.857143,2315.000000,2350.833333,87.962500,-27.158038,1
3,3,41,2019-03-08,2280,2310,2275,2305,80819,3S,2333.75,...,2400.0,2170.0,58.695652,61.594203,44.262295,2296.666667,2340.500000,85.175000,-34.308450,1
4,4,42,2019-03-11,2280,2340,2265,2340,50035,3S,2325.75,...,2400.0,2170.0,73.913043,65.217391,52.307692,2315.000000,2330.833333,81.412500,-12.965522,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374355,426498,511352,2019-12-02,11946,12397,11846,12197,42673,힘스,12260.75,...,13599.0,11420.0,35.658559,32.653975,54.084335,12146.666667,12281.683333,419.104167,-21.477026,1
374356,426499,511353,2019-12-03,12171,12572,11721,12447,51017,힘스,12272.00,...,13599.0,11570.0,43.223263,34.340445,60.095159,12246.666667,12277.100000,414.291667,-4.897248,1
374357,426500,511354,2019-12-04,12572,13248,12547,12748,182189,힘스,12309.55,...,13599.0,11670.0,55.883878,44.921900,55.491463,12847.666667,12317.983333,421.424167,83.792590,1
374358,426501,511355,2019-12-05,12848,12898,12247,12447,66915,힘스,12348.35,...,13599.0,11670.0,40.279938,46.462359,45.728888,12530.666667,12356.400000,399.865833,29.054190,1


In [86]:
Base_data = df

Base_data['BOL_trade'] = 0 # 밴드의 상한 및 하한선 벗어날 때
Base_data['MACD_trade'] = 0 # 음->양 매수 양->음 매도 
Base_data['RSI_trade'] = 0 # 70이상 매도 , 30이하 매수 
Base_data['CCI_trade'] = 0 # -100 ->상향 매수, 100 -> 하향 매도 
Base_data['STO_trade'] = 0 # k가 d를 상향 돌파, 매수 / k가 d를 하향 돌파 매도 
Base_data['MFI_trade'] = 0 # 20이하로 떨어지면 매수 , 80이상으로 가면 매도 

BOL_c = 0
MACD_c = 0
RSI_c = 0
CCI_c = 0
STO_c = 0
MFI_c = 0


# 볼린저밴드 upper, lower 
# macd = MACD_Oscil
# sto = fast_k, slow_d
# RSI
# CCI

Alpha = 0 
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['CCI'][i] < -100:
            Alpha = -1 
    if Alpha == -1:
        if Base_data['CCI'][i] > -100:
            Base_data['CCI_trade'][i] = 1
            CCI_c += 1
            Alpha = 0
    if Alpha == 0:       
        if Base_data['CCI'][i] > 100:
            Alpha = 1
    if Alpha == 1:
        if Base_data['CCI'][i] < 100:
            Base_data['CCI_trade'][i] = 2
            CCI_c += 1
            Alpha = 0
 
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['Close'][i] < Base_data['lower'][i]: # 종가가 하한선보다 작다면 
            Alpha = -1 
    if Base_data['Close'][i] > Base_data['lower'][i]:
        if Alpha == -1:
            Base_data['BOL_trade'][i] = 1
            BOL_c += 1
            Alpha = 0 
            
    if Alpha == 0:
        if Base_data['Close'][i] > Base_data['upper'][i]: # 종가가 하한선보다 크면
            Alpha = 1
    if Base_data['Close'][i] < Base_data['upper'][i]:
        if Alpha == 1:
            Base_data['BOL_trade'][i] = 2
            BOL_c += 1
            Alpha = 0
            
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['MACD_Oscil'][i] < 0:
            Alpha = -1
    if Base_data['MACD_Oscil'][i] > 0:
        if Alpha == -1:
            Base_data['MACD_trade'][i] = 1
            MACD_c += 1
            Alpha = 1 # 0
            
    if Alpha == 0:           
        if Base_data['MACD_Oscil'][i] > 0:
            Alpha = 1
    if Base_data['MACD_Oscil'][i] < 0:
        if Alpha == 1:
            Base_data['MACD_trade'][i] = 2
            MACD_c += 1
            Alpha = -1 # 0
            
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['RSI'][i] < 30:
            Alpha = -1
    if Base_data['RSI'][i] > 30:
        if Alpha == -1:
            Base_data['RSI_trade'][i] = 1
            RSI_c += 1
            Alpha = 0
    if Alpha == 0:           
        if Base_data['RSI'][i] > 70:
            Alpha = 1 
    if Base_data['RSI'][i] < 70:
        if Alpha == 1:
            Base_data['RSI_trade'][i] = 2
            RSI_c += 1
            Alpha = 0
    
Alpha = 0
for i in range(len(Base_data)):
    if Alpha == 0:
        if Base_data['fast_k'][i] < Base_data['slow_d'][i]:
            Alpha = -1
    if Alpha == -1:
        if Base_data['fast_k'][i] > Base_data['slow_d'][i]:
            Base_data['STO_trade'][i] = 1
            STO_c += 1
            Alpha = 1 # 0 
            
    if Alpha == 0:
        if Base_data['fast_k'][i] > Base_data['slow_d'][i]:
            Alpha = 1
    if Alpha == 1:
        if Base_data['fast_k'][i] < Base_data['slow_d'][i]:
            Base_data['STO_trade'][i] = 2
            STO_c += 1
            Alpha =  -1 # 0

Base_data

<ipython-input-86-fe91d4b24ed0>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Base_data['CCI_trade'][i] = 2
<ipython-input-86-fe91d4b24ed0>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Base_data['CCI_trade'][i] = 1
<ipython-input-86-fe91d4b24ed0>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Base_data['BOL_trade'][i] = 2
<ipython-input-86-fe91d4b24ed0>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,level_0,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,...,m,D,CCI,bool,BOL_trade,MACD_trade,RSI_trade,CCI_trade,STO_trade,MFI_trade
0,0,38,2019-03-05,2400,2400,2280,2310,83997,3S,2361.00,...,2366.083333,92.920833,-25.888226,1,0,0,0,0,0,0
1,1,39,2019-03-06,2335,2350,2305,2315,40146,3S,2352.75,...,2358.750000,90.241667,-26.164312,1,0,0,0,0,0,0
2,2,40,2019-03-07,2320,2350,2280,2315,83304,3S,2344.00,...,2350.833333,87.962500,-27.158038,1,0,1,0,0,0,0
3,3,41,2019-03-08,2280,2310,2275,2305,80819,3S,2333.75,...,2340.500000,85.175000,-34.308450,1,0,0,0,0,2,0
4,4,42,2019-03-11,2280,2340,2265,2340,50035,3S,2325.75,...,2330.833333,81.412500,-12.965522,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374355,426498,511352,2019-12-02,11946,12397,11846,12197,42673,힘스,12260.75,...,12281.683333,419.104167,-21.477026,1,0,0,0,0,1,0
374356,426499,511353,2019-12-03,12171,12572,11721,12447,51017,힘스,12272.00,...,12277.100000,414.291667,-4.897248,1,0,0,0,0,0,0
374357,426500,511354,2019-12-04,12572,13248,12547,12748,182189,힘스,12309.55,...,12317.983333,421.424167,83.792590,1,0,0,0,0,0,0
374358,426501,511355,2019-12-05,12848,12898,12247,12447,66915,힘스,12348.35,...,12356.400000,399.865833,29.054190,1,0,2,0,0,2,0


In [87]:
Base_data['MACD_Oscil'].mean()

-11.4459946800211

In [88]:
Base_data[Base_data['종목명'] == '3S']

,level_0,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,...,m,D,CCI,bool,BOL_trade,MACD_trade,RSI_trade,CCI_trade,STO_trade,MFI_trade
0,0,38,2019-03-05,2400,2400,2280,2310,83997,3S,2361.00,...,2366.083333,92.920833,-25.888226,1,0,0,0,0,0,0
1,1,39,2019-03-06,2335,2350,2305,2315,40146,3S,2352.75,...,2358.750000,90.241667,-26.164312,1,0,0,0,0,0,0
2,2,40,2019-03-07,2320,2350,2280,2315,83304,3S,2344.00,...,2350.833333,87.962500,-27.158038,1,0,1,0,0,0,0
3,3,41,2019-03-08,2280,2310,2275,2305,80819,3S,2333.75,...,2340.500000,85.175000,-34.308450,1,0,0,0,0,2,0
4,4,42,2019-03-11,2280,2340,2265,2340,50035,3S,2325.75,...,2330.833333,81.412500,-12.965522,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,186,224,2019-12-02,2905,2945,2855,2890,146933,3S,3121.50,...,3129.166667,131.958333,-117.461320,1,0,0,0,0,0,0
187,187,225,2019-12-03,2890,2975,2835,2970,146397,3S,3103.75,...,3108.416667,139.600000,-86.795606,1,0,0,1,1,1,0
188,188,226,2019-12-04,2960,2985,2910,2925,76872,3S,3087.50,...,3092.000000,142.829167,-70.947227,1,0,0,0,0,2,0
189,189,227,2019-12-05,2930,2940,2795,2800,160164,3S,3066.00,...,3071.750000,149.729167,-100.960067,1,0,0,0,0,0,0


In [89]:
print(len(Base_data[Base_data['MACD_trade'] == 1]))
print(len(Base_data[Base_data['MACD_trade'] == 2]))

print(len(Base_data[Base_data['STO_trade'] == 1]))
print(len(Base_data[Base_data['STO_trade'] == 2]))

print(len(Base_data[Base_data['RSI_trade'] == 1]))
print(len(Base_data[Base_data['RSI_trade'] == 2]))

print(len(Base_data[Base_data['CCI_trade'] == 1]))
print(len(Base_data[Base_data['CCI_trade'] == 2]))

print(len(Base_data[Base_data['BOL_trade'] == 1]))
print(len(Base_data[Base_data['BOL_trade'] == 2]))

12803
12803
73121
73121
14734
10302
13131
10618
11324
9376


In [90]:
len(b)

191

In [91]:
trade_data = Base_data

trade_data['BOL_profit']= 0
trade_data['MACD_profit']= 0
trade_data['RSI_profit']= 0
trade_data['CCI_profit']= 0
trade_data['STO_profit']= 0

trade_data['MFI_profit'] = 0

trade_data['Ori']= 0


# 한 종목 개수 
len_a = len(b)
start_m = 30000000 # 시작 돈 
plus_m = 0 # 월급 

# 0-대기 1-매수 2-매도
def trade(data, asset, indicator):
    day = 0
    ori = 0
    count = 0 
    can_b = 0
    can_s = 0
    for _ in range(len(data)-1):
        if _%len_a == 0:
            day = 0
            ori = 0
            count = 0 
            can_b = 0
            can_s = 0
            asset = start_m
        if day != 0:
            if day%30 == 0:
                ori += 1
                day = 0 
                asset += plus_m
                
        if data[indicator + '_trade'][_] == 0:
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
        elif data[indicator + '_trade'][_] == 1:
            can_b = int(asset / data['Open'][_+1])
            asset -= can_b*data['Open'][_+1]
            count += can_b 
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
            can_b = 0
        elif data[indicator + '_trade'][_] == 2:
            can_s = int(count)
            asset += can_s*data['Open'][_+1]
            count -= can_s
            data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
            can_s = 0
        data['Ori'][_] = start_m + (ori)*plus_m
        day += 1
    return data

trade_data = trade(trade_data, start_m, "BOL")
trade_data = trade(trade_data, start_m, "MACD")
trade_data = trade(trade_data, start_m, "RSI")
trade_data = trade(trade_data, start_m, "CCI")
trade_data = trade(trade_data, start_m, "STO")

trade_data = trade(trade_data, start_m, "MFI")

trade_data

<ipython-input-91-75f308389ac4>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
<ipython-input-91-75f308389ac4>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Ori'][_] = start_m + (ori)*plus_m
<ipython-input-91-75f308389ac4>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[indicator + '_profit'][_] = asset + (count*data['Close'][_])
<ipython-input-91-75f

,level_0,index,날짜,Open,High,Low,Close,Volume,종목명,ma20,...,CCI_trade,STO_trade,MFI_trade,BOL_profit,MACD_profit,RSI_profit,CCI_profit,STO_profit,MFI_profit,Ori
0,0,38,2019-03-05,2400,2400,2280,2310,83997,3S,2361.00,...,0,0,0,30000000,30000000,30000000,30000000,30000000,30000000,30000000
1,1,39,2019-03-06,2335,2350,2305,2315,40146,3S,2352.75,...,0,0,0,30000000,30000000,30000000,30000000,30000000,30000000,30000000
2,2,40,2019-03-07,2320,2350,2280,2315,83304,3S,2344.00,...,0,0,0,30000000,30460495,30000000,30000000,30000000,30000000,30000000
3,3,41,2019-03-08,2280,2310,2275,2305,80819,3S,2333.75,...,0,2,0,30000000,30328925,30000000,30000000,30000000,30000000,30000000
4,4,42,2019-03-11,2280,2340,2265,2340,50035,3S,2325.75,...,0,1,0,30000000,30789420,30000000,30000000,29311070,30000000,30000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374355,426498,511352,2019-12-02,11946,12397,11846,12197,42673,힘스,12260.75,...,0,1,0,61224356,48673403,54598622,51986646,33115054,30000000,30000000
374356,426499,511353,2019-12-03,12171,12572,11721,12447,51017,힘스,12272.00,...,0,0,0,61224356,49670903,55717622,51986646,33793804,30000000,30000000
374357,426500,511354,2019-12-04,12572,13248,12547,12748,182189,힘스,12309.55,...,0,0,0,61224356,50871893,57064898,51986646,34611019,30000000,30000000
374358,426501,511355,2019-12-05,12848,12898,12247,12447,66915,힘스,12348.35,...,0,2,0,61224356,49471403,55717622,51986646,33658054,30000000,30000000


In [92]:
data = trade_data

len_a = (len(data[data['종목명'] == data['종목명'][5]]))
print(len_a)

win = 0
lose = 0
state = 0
b = 0
s = 0

b_c = 0
m_c = 0
r_c = 0
c_c = 0
s_c = 0

for i in range(len(data)):
    if i%len_a == 0:
        state = 0
    if state==0:
        if data['BOL_trade'][i] == 1:
            b = data['Close'][i]
            state = 1
    if state== 1:
        if data['BOL_trade'][i] == 2:
            s = data['Close'][i]  
            state = 0
            b_c += 1 
            if b < s:
                win +=1
            else:
                lose+=1
            # print(b,s)
            b=0
            s=0
        elif data['BOL_trade'][i] == 1:
            # b = data['Close'][i]
            # lose += 1
            state = 1
print(win/(win+lose))

win = 0
lose = 0
state = 0
b = 0
s = 0

for i in range(len(data)):
    if i%len_a == 0:
        state = 0
    if state==0:
        if data['MACD_trade'][i] == 1:
            b = data['Close'][i]
            state = 1
    if state== 1:
        if data['MACD_trade'][i] == 2:
            s = data['Close'][i]  
            state = 0
            m_c += 1 
            if b < s:
                win +=1
            else:
                lose+=1
            b=0
            s=0
        elif data['MACD_trade'][i] == 1:
            # b = data['Close'][i]
            # lose += 1
            state = 1
print(win/(win+lose))

win = 0
lose = 0
state = 0
b = 0
s = 0

for i in range(len(data)):
    if i%len_a == 0:
        state = 0
    if state==0:
        if data['RSI_trade'][i] == 1:
            b = data['Close'][i]
            state = 1
    if state== 1:
        if data['RSI_trade'][i] == 2:
            s = data['Close'][i]  
            state = 0
            r_c += 1 
            if b < s:
                win +=1
            else:
                lose+=1
            b=0
            s=0
        elif data['RSI_trade'][i] == 1:
            # b = data['Close'][i]
            # lose += 1
            state = 1
print(win/(win+lose))

win = 0
lose = 0
state = 0
b = 0
s = 0

for i in range(len(data)):
    if i%len_a == 0:
        state = 0
    if state==0:
        if data['CCI_trade'][i] == 1:
            b = data['Close'][i]
            state = 1
    if state== 1:
        if data['CCI_trade'][i] == 2:
            s = data['Close'][i]  
            state = 0
            c_c += 1 
            if b < s:
                win +=1
            else:
                lose+=1
            b=0
            s=0
        elif data['CCI_trade'][i] == 1:
            # b = data['Close'][i]
            # lose += 1
            state = 1
print(win/(win+lose))



win = 0
lose = 0
state = 0
b = 0
s = 0

for i in range(len(data)):
    if i%len_a == 0:
        state = 0
        
    if state==0:
        if data['STO_trade'][i] == 1:
            b = data['Close'][i]
            state = 1
    if state== 1:
        if data['STO_trade'][i] == 2:
            s = data['Close'][i]  
            state = 0
            s_c += 1 
            if b < s:
                win +=1
            else:
                lose+=1
            b=0
            s=0
        elif data['STO_trade'][i] == 1:
            # b = data['Close'][i]
            # lose += 1
            state = 1
print(win/(win+lose))

191
0.6019063638912251
0.40001613293538757
0.5945263157894737
0.5929304446978335
0.3353473898664583


In [93]:
# 페어 형성된 개수 
print(b_c,m_c,r_c,c_c,s_c)

3567 12397 4750 4385 71663


In [97]:
len(trade_data)

374360

In [94]:
print(len(data[data['BOL_trade']  == 1]), len(data[data['BOL_trade']  == 2]), "BOL",1/((len(data[data['BOL_trade']  == 1])+len(data[data['BOL_trade']  == 2]))/len(data)))


print(len(data[data['MACD_trade']  == 1]), len(data[data['MACD_trade']  == 2]),"MACD",1/((len(data[data['MACD_trade']  == 1])+len(data[data['MACD_trade']  == 2]))/len(data)))


print(len(data[data['RSI_trade']  == 1]), len(data[data['RSI_trade']  == 2]),"RSI",1/((len(data[data['RSI_trade']  == 1])+len(data[data['RSI_trade']  == 2]))/len(data)))


print(len(data[data['CCI_trade']  == 1]), len(data[data['CCI_trade']  == 2]),"CCI",1/((len(data[data['CCI_trade']  == 1])+len(data[data['CCI_trade']  == 2]))/len(data)))


print(len(data[data['STO_trade']  == 1]), len(data[data['STO_trade']  == 2]),"STO",1/((len(data[data['STO_trade']  == 1])+len(data[data['STO_trade']  == 2]))/len(data)))

11324 9376 BOL 18.085024154589373
12803 12803 MACD 14.620010934937122
14734 10302 RSI 14.952867870266816
13131 10618 CCI 15.763190029053856
73121 73121 STO 2.559866522613203


In [95]:
# 맞췄을까? 

trade_data['BOL_bool'] = 0
trade_data['MACD_bool'] = 0
trade_data['RSI_bool'] = 0
trade_data['CCI_bool'] = 0
trade_data['STO_bool'] = 0

trade_data['trend'] = 0 

for _ in range(1, len(trade_data)):
    if _%len_a == 0:
        continue
    if trade_data['Close'][_] - trade_data['Close'][_-1] == 0:
        trade_data['trend'][_] = 0
    elif trade_data['Close'][_] - trade_data['Close'][_-1] > 0:
        trade_data['trend'][_] = 1
    elif trade_data['Close'][_] - trade_data['Close'][_-1] < 0:
        trade_data['trend'][_] = -1
        
# trend가 나왔고, 
def gogo(data, indicator):
    state = 0
    for _ in range(len(data)-1):
        if _%len_a == 0:
            state = 0
        # state check
        if data[indicator + '_trade'][_] == 1:
            state = 1 # 매수 상태 
        elif data[indicator + '_trade'][_] == 2:
            state = 2 # 매도 상태
        # state 별로 값 할당 
        if state == 0:
            if data[indicator + '_trade'][_] == 0:
                data[indicator + '_bool'][_] = state
        elif state == 1: # 매수 상태
            # 1 or -1
            if data['trend'][_] >=0
                data[indicator + '_bool'][_] = 1
            else:
                data[indicator + '_bool'][_] = -1
        elif state == 2: # 매도 상태 
            if data['trend'][_] <= 0 :
                data[indicator + '_bool'][_] = 1
            else:
                data[indicator + '_bool'][_] = -1
    return data

gogo(trade_data, 'BOL')
gogo(trade_data, 'MACD')
gogo(trade_data, 'RSI')
gogo(trade_data, 'CCI')
gogo(trade_data, 'STO')


# 1이 맞춘거, -1이 틀린것 

print(len(trade_data[trade_data['BOL_bool'] == 1]))
print(len(trade_data[trade_data['BOL_bool'] == -1]))

print(len(trade_data[trade_data['MACD_bool'] == 1]))
print(len(trade_data[trade_data['MACD_bool'] == -1]))

print(len(trade_data[trade_data['RSI_bool'] == 1]))
print(len(trade_data[trade_data['RSI_bool'] == -1]))

print(len(trade_data[trade_data['CCI_bool'] == 1]))
print(len(trade_data[trade_data['CCI_bool'] == -1]))

print(len(trade_data[trade_data['STO_bool'] == 1]))
print(len(trade_data[trade_data['STO_bool'] == -1]))




<ipython-input-95-e4ec70f2982e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_data['trend'][_] = 1
<ipython-input-95-e4ec70f2982e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_data['trend'][_] = 0
<ipython-input-95-e4ec70f2982e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade_data['trend'][_] = -1
<ipython-input-95-e4ec70f2982e>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

196063
151950
200663
160514
199151
155681
199130
156597
309939
62272


In [113]:
# 오른 시점은 뺴야함 
BOL_trade = trade_data[trade_data['BOL_trade'] == 0]
print(len(BOL_trade[BOL_trade['BOL_bool'] == 1]))
print(len(BOL_trade[BOL_trade['BOL_bool'] == -1]))
print(len(trade_data[trade_data['BOL_trade'] == 0]))

MACD_trade = trade_data[trade_data['MACD_trade'] == 0]
print(len(MACD_trade[MACD_trade['MACD_bool'] == 1]))
print(len(MACD_trade[MACD_trade['MACD_bool'] == -1]))
print(len(trade_data[trade_data['MACD_trade'] == 0]))

RSI_trade = trade_data[trade_data['RSI_trade'] == 0]
print(len(RSI_trade[RSI_trade['RSI_bool'] == 1]))
print(len(RSI_trade[RSI_trade['RSI_bool'] == -1]))
print(len(trade_data[trade_data['RSI_trade'] == 0]))

CCI_trade = trade_data[trade_data['CCI_trade'] == 0]
print(len(CCI_trade[CCI_trade['CCI_bool'] == 1]))
print(len(CCI_trade[CCI_trade['CCI_bool'] == -1]))
print(len(trade_data[trade_data['CCI_trade'] == 0]))

STO_trade = trade_data[trade_data['STO_trade'] == 0]
print(len(STO_trade[STO_trade['STO_bool'] == 1]))
print(len(STO_trade[STO_trade['STO_bool'] == -1]))
print(len(trade_data[trade_data['STO_trade'] == 0]))


177537
149776
353660
183897
151674
348754
177124
152672
349324
178472
153506
350611
177602
48368
228118


In [159]:
# 선행지표로서의 승률 
JM = len(trade_data[trade_data['종목명'] == trade_data['종목명'][0]])

BOL_index1 = trade_data.index[trade_data['BOL_trade'] == 1]
BOL_index2 = trade_data.index[trade_data['BOL_trade'] == 2]
BOL_index1 = BOL_index1.to_list()
BOL_index2 = BOL_index2.to_list()

BOL_indexs = BOL_index1 + BOL_index2

BOL_count1 = 0
BOL_count2 = 0
for i in BOL_indexs:
    if i%JM == 0:
        #print("pass")
        continue
    if i == len(trade_data):
        break
    if trade_data['BOL_bool'][i+1] == 1:
        BOL_count1 += 1
    elif trade_data['BOL_bool'][i+1] == -1:
        BOL_count2 += 1 
print(BOL_count1, BOL_count2, BOL_count1/(BOL_count1+BOL_count2))

# 선행지표로서의 승률 
MACD_index1 = trade_data.index[trade_data['MACD_trade'] == 1]
MACD_index2 = trade_data.index[trade_data['MACD_trade'] == 2]
MACD_index1 = MACD_index1.to_list()
MACD_index2 = MACD_index2.to_list()

MACD_indexs = MACD_index1 + MACD_index2

MACD_count1 = 0
MACD_count2 = 0
for i in MACD_indexs:
    if i%JM == 0:
        #print("pass")
        continue
    if i == len(trade_data):
        break
    if trade_data['MACD_bool'][i+1] == 1:
        MACD_count1 += 1
    elif trade_data['MACD_bool'][i+1] == -1:
        MACD_count2 += 1 
print(MACD_count1, MACD_count2, MACD_count1/(MACD_count1+MACD_count2))

# 선행지표로서의 승률 
RSI_index1 = trade_data.index[trade_data['RSI_trade'] == 1]
RSI_index2 = trade_data.index[trade_data['RSI_trade'] == 2]
RSI_index1 = RSI_index1.to_list()
RSI_index2 = RSI_index2.to_list()

RSI_indexs = RSI_index1 + RSI_index2

RSI_count1 = 0
RSI_count2 = 0
for i in RSI_indexs:
    if i%JM == 0:
        #print("pass")
        continue
    if i == len(trade_data):
        break
    if trade_data['RSI_bool'][i+1] == 1:
        RSI_count1 += 1
    elif trade_data['RSI_bool'][i+1] == -1:
        RSI_count2 += 1 
print(RSI_count1, RSI_count2, RSI_count1/(RSI_count1+RSI_count2))

# 선행지표로서의 승률 
CCI_index1 = trade_data.index[trade_data['CCI_trade'] == 1]
CCI_index2 = trade_data.index[trade_data['CCI_trade'] == 2]
CCI_index1 = CCI_index1.to_list()
CCI_index2 = CCI_index2.to_list()

CCI_indexs = CCI_index1 + CCI_index2

CCI_count1 = 0
CCI_count2 = 0
for i in CCI_indexs:
    if i%JM == 0:
        #print("pass")
        continue
    if i == len(trade_data):
        break
    if trade_data['CCI_bool'][i+1] == 1:
        CCI_count1 += 1
    elif trade_data['CCI_bool'][i+1] == -1:
        CCI_count2 += 1 
print(CCI_count1, CCI_count2, CCI_count1/(CCI_count1+CCI_count2))

# 선행지표로서의 승률 
# 1과 2 겹치는건? 
STO_index1 = trade_data.index[trade_data['STO_trade'] == 1]
STO_index2 = trade_data.index[trade_data['STO_trade'] == 2]
STO_index1 = STO_index1.to_list()
STO_index2 = STO_index2.to_list()

STO_indexs = STO_index1 + STO_index2

STO_count1 = 0
STO_count2 = 0

for _ in STO_indexs:
    if _%JM == 0:
        #print("pass")
        continue
    if _+1 == len(trade_data):
        break
        
    if trade_data['STO_bool'][_+1] == 1:
        STO_count1 += 1
    elif trade_data['STO_bool'][_+1] == -1:
        STO_count2 += 1 
        
    #if trade_data['']
        
print(STO_count1, STO_count2, STO_count1/(STO_count1+STO_count2))



12001 8396 0.5883708388488503
13369 11372 0.5403581100198052
12529 11746 0.5161277033985582
12374 10680 0.5367398282293745
58856 13403 0.8145144549467886


In [155]:
# 수익률 가져다 붙이기 

[4, 15, 21, 23, 31, 33, 38, 42, 49, 54]